# Kubernetes Configuration

> `Kubernetes`, being as large as it is, __provides a few ways to configure various components via... yes, `.yaml` files.__

# ConfigMaps

> __API object used to store non-confidential data in key-value pairs__

These can be consumed by `POD`s via:
- `envvars`
- command line
- __config files in a `volume`__

Why should we use them?

> __Allows us to decouple environment specific configs from our container `images`__

For example, our code might look for environment variable `DATABASE_HOST` which:
- should be `localhost` for our development
- should be `<SOME-IP:SOME-PORT>` for deployment

Above can be set in aforementioned (up to `1Mb`, __which is plenty for most config files__).

__Example config file__:

In [ ]:
apiVersion: v1
kind: ConfigMap
metadata:
  name: game-demo
data:
  # property-like keys; each key maps to a simple value
  player_initial_lives: "3"
  ui_properties_file_name: "user-interface.properties"

  # file-like keys
  game.properties: |
    enemy.types=aliens,monsters
    player.maximum-lives=5    
  user-interface.properties: |
    color.good=purple
    color.bad=yellow
    allow.textmode=true  

Things to note:
- __no `.spec`!__
- `.data` and/or `.binaryData` fields
- Multiline nested values one can later use (simulating `file` structure)

Above can be consumed by `POD`s `config` file (__you should prefer this approach to consuming config files as it fits well with `kubectl apply`!__):

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: configmap-demo-pod
spec:
  containers:
    - name: demo
      image: alpine
      command: ["sleep", "3600"]
      env:
        # Define the environment variable (should be uppercase)
        - name: PLAYER_INITIAL_LIVES 
          # Where to take this envvar valueFrom
          valueFrom:
            # From configMapKeyRef(erence)
            configMapKeyRef:
              # Name of `ConfigMap`
              name: game-demo          
              key: player_initial_lives # The key to fetch.
        - name: UI_PROPERTIES_FILE_NAME
          valueFrom:
            configMapKeyRef:
              name: game-demo
              key: ui_properties_file_name
      # Mount ephemeral volume for config (POD life duration)
      volumeMounts:
      - name: config
        mountPath: "/config"
        readOnly: true
  volumes:
    # You set volumes at the Pod level, then mount them into containers inside that Pod
    - name: config
      configMap:
        # Provide the name of the ConfigMap you want to mount.
        name: game-demo
        # An array of keys from the ConfigMap to create as files
        items:
        - key: "game.properties"
          path: "game.properties"
        - key: "user-interface.properties"
          path: "user-interface.properties"

Please notice that:
- __Files will be automatically created from `ConfigMap`!__
- Once they are within `POD` __we define how to consume them__ (e.g. reading `game.properties` mounted file from our `container`)
- __If you omit the items array entirely__, every key in the ConfigMap becomes a file with the same name as the key, __and you get 4 files.__

__Important features:__
- __IF WE UPDATE `ConfigMap` THESE WILL BE AUTOMATICALLY UPDATED FOR OUR CONTAINERS TO READ FROM!__
- Other parts of the program can use these values however we wish (as long as they are located in the same `POD`)
- __One can also set the `ConfigMap`s to be immutable__ via `.immutable=true` flag (in this case __it won't be automatically updated__)

Another example of mounting `ConfigMap`:

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: mypod
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    configMap:
      name: myconfigmap

In general:
- Each `ConfigMap` needs it's respective `.spec.volumes` entry
- Each container needs one `.spec.containers.volumeMounts` entry __per `ConfigMap`__
- __As many `ConfigMap`s we need, we can create them__

# Secrets

> `Secrets` allow us to safely provide sensitive data for our application (e.g. `ssh` keys, logins, passwords etc.)

Why should we do that?
- __No way to inspect `container`'s `image` for secrets__
- Not provided within the code
- __It DOES NOT save us from revealing the secrets;__ one can still `print` them by accident, hence, as always __look out for revealing these parts!__

In general:
- Similar to `ConfigMaps`
- Created independently from `POD`s
- Additional precautions available like:
    - Not writing to non-volatile memory (e.g. `disk` where one could read `secret`s from)
    - Encrypting `secret`s at rest (see __Challenges__ at the end of the notebook __AND USE IT!__)
- Once again: __not encrypted by default__ we need to set it up separately (yes, new `k8s` object defined with `.yaml`)
- __One has to provide them as `base64` encoded strings__ (please notice: `base64` __IS NOT AN ENCRYPTION__ and should be regarded as provided in plain text)
- __Created as resources__

> __Secrets are stored by `etcd` in `base64` encoding, hence you really should set up a so called "encryption at rest"__

## Types of `Secrets`

> `k8s` provides various types of secrets for different use cases.

There are:
- built-in ones for common use cases
- __we can define our own `type`s__

Most common ones:

- __Opaque Secret__ - __arbitrary `secret` data__; Default type (if ommited) of a `Secret`
- `kubernetes.io/service-account-token` - stores a token identifying `Service` account

Second one could be defined like this:

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: secret-sa-sample
  annotations:
    kubernetes.io/service-account.name: "sa-name"
type: kubernetes.io/service-account-token
data:
  # You can include additional key value pairs as you do with Opaque Secrets
  extra: YmFyCg==

> __When creating a Pod, Kubernetes automatically creates a service account Secret and automatically modifies your `Pod` to use this `Secret`. The service account token Secret contains credentials for accessing the API.__

See [this part of documentation](https://kubernetes.io/docs/tasks/configure-pod-container/configure-service-account/) for more information.

- `kubernetes.io/dockerconfigjson` - store a serialized `docker` configuration to be used by `POD` (e.g. `~/.docker/config.json` file)

For example:

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: secret-dockercfg
type: kubernetes.io/dockerconfigjson
data:
  .dockercfg: |
        "<base64 encoded ~/.docker/config.json file>"

Traits:
- Validated by `API` whether it can be decoded as valid `JSON`
- __Does not validate whether it is a correct `Docker` configuration `.json` file__

### Basic Auth

> __This type of secret is used for `username` and `password` storing__

This one is the most common, but is __as easy as ever to use__:

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: secret-basic-auth
type: kubernetes.io/basic-auth
stringData:
  username: admin
  password: t0p-Secret
# Immutable also supported
immutable: true

> If we don't want to `base64` encode our `data` field, __we can use `stringData` instead!__

### ssh-secrets

As was the case previously, we might use `Opaque` secret type, but:
- We would need to define our own structure of fields
- __These WOULD NOT be verified__ (whether appropriate fields where added)

Due to the above, __you should stick to provided `types` if possible__

In [ ]:
apiVersion: v1
kind: Secret
metadata:
  name: secret-ssh-auth
type: kubernetes.io/ssh-auth
data:
  # the data is abbreviated in this example
  ssh-privatekey: |
          MIIEpQIBAAKCAQEAulqb/Y ...

## Using `Secrets`

> __This type of secret is used during `Node` bootstrap process!__

One can:
- Mount them as data volumes
- Expose as `envvars` (see [here](https://kubernetes.io/docs/concepts/configuration/secret/#using-secrets-as-environment-variables) for information)

As with `config` files one should be prefer mounting as:
- It allows us to update `secrets` on the fly with `kubectl apply -R -f <dir>`
- __Coherent usage with `config`s__ (less mental overhead)

Let's see, how one can use them for `POD`s (__do not use "bare `POD`s, use `workloads` instead!__):

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: secret-env-pod
spec:
  containers:
  - name: mycontainer
    image: redis
    env:
      - name: SECRET_USERNAME
        valueFrom:
          secretKeyRef:
            name: mysecret
            key: username
      - name: SECRET_PASSWORD
        valueFrom:
          secretKeyRef:
            name: mysecret
            key: password
  restartPolicy: Never

> __Referencing keys works the same as with `config`, e.g.: these are loaded as environment variables!__

In the above case we will have `2` `envvars`:
- `SECRET_USERNAME`
- `SECRET_PASSWORD`

We can use them from within our containers in our code.

# Managing Resources

> ### One can specify various requirements (`RAM`, `CPU` or others) for `container`s within `POD`s

There are two types of constraints one can specify:
- `requests` - __at least this amount is needed__; used when scheduling `POD`s to different `Node`s
- `limits` - __at most this amount will be used by `POD`__; as above + limits the resources available

This section is pretty important as it allows us to "help `k8s`" to:
- __efficiently schedule `POD`s across `Node`s__ (simplified example: if it's known that our container will use at most `1Gb` of RAM and we have `8Gb` available, `~8` `POD`s might be scheduled)

Limits can be set:
- `reactively` - when `container` uses too much resources system intervenes
- by enforcement - when `container` tries too use too much resources __it is killed__

> We will stick to `RAM` and `CPU` as it is the most common resource we would like to control

One can also `request` and `limit` `GPU`s (for deep learning oriented applications). __See [here](https://kubernetes.io/docs/tasks/manage-gpus/scheduling-gpus/) for details__

## Rules

Some rules governing scheduling:

- If a `Container` exceeds its memory limit, it might be terminated. If it is restartable, the `kubelet` will restart it, as with any other type of runtime failure.
- If a `Container` exceeds its memory request, it is likely that its Pod will be evicted whenever the node runs out of memory.
- A `Container` might or might not be allowed to exceed its CPU limit for extended periods of time. However, it will not be killed for excessive `CPU` usage.

> __Above IS NOT user controlled, hence some issues may arise. Refer to [Troubleshooting](https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/#troubleshooting) in case you suspect resource limits are to blame__


## Defining

- __Do it per-container__
- Specify them within `resources` field, which can have:
    - `requests` field
    - `limits` field
    
Let's see an example:

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: frontend
spec:
  containers:
  - name: app
    image: images.my-company.example/app:v4
    resources:
      requests:
        memory: "64Mi"
        cpu: "250m"
      limits:
        memory: "128Mi"
        cpu: "500m"
  - name: log-aggregator
    image: images.my-company.example/log-aggregator:v6
    resources:
      requests:
        memory: "64Mi"
        cpu: "250m"
      limits:
        memory: "128Mi"
        cpu: "500m"

# Challenges

> ### Go through these challenges after whole `kubernetes` content is covered!

## Mandatory

- How to store our secrets? Best way to do that, would be to use `GitHub` (together with other cluster configuration files), __but we don't want others to see credentials__. Check [this blog post](https://learnk8s.io/kubernetes-secrets-in-git) which allows us to do it simply via assymetric encryption (and other means). Also [this repository](https://github.com/bitnami-labs/sealed-secrets) and [this one](https://github.com/Soluto/kamus). Any other ways you can think of storing them in `github` safely?
- Read [Configuration Best Practices](https://kubernetes.io/docs/concepts/configuration/overview/) __and stick to them all the time!__
- Check out [Use Cases of `secrets`](https://kubernetes.io/docs/concepts/configuration/secret/#use-cases)
- Check out [Encrypting `secrets` at rest](https://kubernetes.io/docs/tasks/administer-cluster/encrypt-data/) __and use it all the time!__
- Check out [Local Ephemeral Storage](https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/#local-ephemeral-storage). What is it, how could we utilize it's functionalities?
- Check out [this part of docs](https://kubernetes.io/docs/concepts/configuration/secret/#using-imagepullsecrets) to know how to `pull` images from our private registries with secrets.
- Check out [Local Ephemeral Storage](https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/#local-ephemeral-storage) and how to request it.

## Additional

- We can define how to spot `ConfigMap` updates (default: by watching directory). What are the other ways? Check out [this short intro](https://kubernetes.io/docs/concepts/configuration/configmap/#mounted-configmaps-are-updated-automatically)
- Check out [HashiCorp's Vault](https://www.vaultproject.io/) project __as a better alternative__ for managing secrets. See [this guide](https://www.vaultproject.io/docs/platform/k8s) to get started and follow the links there.
- Check [`RBAC rules`](https://kubernetes.io/docs/reference/access-authn-authz/authorization/). Whu should we use them with `Secrets`? 
- Check [`TLS Secrets`](https://kubernetes.io/docs/concepts/configuration/secret/#tls-secrets) for `Ingress`.
- What are [`Bootstrap Secrets`](https://kubernetes.io/docs/concepts/configuration/secret/#bootstrap-token-secrets)? Should we use them? If so, what are the use cases?
- Check out [`kubeconfig`](https://kubernetes.io/docs/concepts/configuration/organize-cluster-access-kubeconfig/) as a way to manage "general" (non-app specific) configuration